In [ ]:
from DeepFMKit.core import DeepFitFramework
from DeepFMKit.plotting import default_rc
from DeepFMKit.experiments import Experiment
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update(default_rc)

In [ ]:
dff = DeepFitFramework(raw_file='../test/raw_data.txt', raw_labels=['raw1'])
dff.fit('raw1', n=20)

In [ ]:
axes = dff.plot()
plt.show()

In [ ]:
axs = dff.plot(which=['phi','m'], figsize=(5,2), lw=1, c='tomato')
plt.show()

In [ ]:
ax = dff.raws['raw1'].plot(figsize=(5,3), xrange=[0,0.01], c='lime')
ax.grid(False)
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter
from DeepFMKit import physics
from DeepFMKit.helpers import set_laser_df_for_effect

cycles = 1
f_samp = 200e3
f_mod = 1e3
m = 7.2

laser = physics.LaserConfig()
laser.f_mod = f_mod

ifo = physics.InterferometerConfig()
ifo.ref_arml = 0.15
ifo.meas_arml = 0.2
ifo.phi = 0.0

main = physics.DFMIObject(label='Hello', laser_config=laser, ifo_config=ifo, f_samp=f_samp)

set_laser_df_for_effect(laser, ifo, m)

laser.psi = np.pi/3
ifo.phi = np.pi/3

ax = main.plot_harmonics(N=10)
plt.show()

In [ ]:
sg = physics.SignalGenerator()

fig, ax = plt.subplots(figsize=(3.375, 1.8), dpi=150)
raw = sg.generate(main, n_seconds=cycles/f_mod, mode='snr', snr_db=100)['main']
ax = raw.plot(ax=ax, ls='-', lw=1.0)

ax.set_ylabel('Voltage (a.u.)')
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x * 1e3:.1f}"))
ax.set_xlabel('Time (ms)')
ax.grid(False)

plt.show()